## Imports

In [14]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from gpt import GPT
from intent_model import IntentModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [15]:
# Load the data from JSON file
with open("../intent_recognition.json", "r") as f:
    data = json.load(f)

train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])
    

## Load models

In [16]:
intent_model = IntentModel(train_data, label_map, labels)

## Get evaluation data

In [17]:
evaluation_file = "../evaluation_results.json"

if os.path.isfile(evaluation_file):
    with open(evaluation_file, "r") as file:
        accuracies = json.load(file)
else:
    accuracies = intent_model.evaluate_model(data)

## Check data completeness

In [18]:
from collections import defaultdict

# Your data is already a list of dictionaries
categories = defaultdict(list)

for item in data:
    intent_text = item['intent']
    if not item['use_gpt'] and not item['responses']:

        intent_text += ' (Needs answers)'
    categories[item['category']].append(intent_text)

for category, intents in categories.items():
    if(category == 'privacy_policy' or category ==  'legal_statement'):
        continue

    print(f'Category: {category}')
    for intent in intents:
        print(f'  Intent: {intent}')
    print('\n')

Category: platform_settings
  Intent: location_password_change
  Intent: reset_password_change
  Intent: update_password_change
  Intent: problem_update_password_change


Category: My_visibility_settings
  Intent: visibility_publish_profile_status
  Intent: visibility_publish_profile_how_to


Category: system notifications
  Intent: change_system_critical_notification_setting
  Intent: change_iYYU_news_notification_setting
  Intent: change_update_notification_setting


Category: connect
  Intent: change_note_follower_notification_setting
  Intent: change_connection_response_notification_setting
  Intent: change_connection_request_notification_setting


Category: interact
  Intent: change_followed_user_note_notification_setting
  Intent: change_message_notification_preference


Category: generic
  Intent: yes_response
  Intent: no_response




## Check data correctness

In [19]:
for evaluation in accuracies:
    if (evaluation["total_correctness"] == 100):
        continue

    print(f"Intent: {evaluation['correct_intent']}")
    print(f"Train question results: {evaluation['train_questions_results']}")

    test_results = evaluation.get('test_questions_results')
    if test_results:
        print(f"Test question results: {test_results}")
        
    advanced_results = evaluation.get('test_questions_advanced_results')
    if advanced_results:
        print(f"Advanced test question results: {advanced_results}")
    
    print(f"Total correctness for all questions: {evaluation['total_correctness']}%")
    print("\n")



Intent: third_parties
Train question results: 5 out of 6 correct
Total correctness for all questions: 83%


Intent: data_collection
Train question results: 4 out of 6 correct
Total correctness for all questions: 67%


Intent: personal_information_rights
Train question results: 9 out of 10 correct
Total correctness for all questions: 90%


Intent: question_answering
Train question results: 7 out of 10 correct
Total correctness for all questions: 70%


Intent: location_password_change
Train question results: 9 out of 10 correct
Total correctness for all questions: 90%


Intent: update_password_change
Train question results: 8 out of 13 correct
Total correctness for all questions: 62%


Intent: change_iYYU_news_notification_setting
Train question results: 10 out of 10 correct
Test question results: 4 out of 5 correct
Advanced test question results: 5 out of 6 correct
Total correctness for all questions: 90%


Intent: change_connection_response_notification_setting
Train question results: 